In [1]:
import cv2
from PIL import Image
import glob, os
import json

In [8]:
# Need to convert the Cross-Recessed-Screw data from individual xml labels into a combined coco Json format
# Run that script seperately if the conversion is needed

In [4]:
def norm2abs(bbx_coords, image_shape):
    x, y, w, h, = bbx_coords

    # get x and y
    x_reg = int(x * image_shape[1])
    y_reg = int(y * image_shape[0])
    w_reg = int(w * image_shape[1])
    h_reg = int(h * image_shape[0])

    bbx_coords_reg = [x_reg, y_reg, w_reg, h_reg]
    return bbx_coords_reg

In [13]:
# Check image dimensions, bounding box coordinates and labels
# We will compare the normalised and absolute coordinates, we assume the two json files are keyed identically
root = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/'
directory = root + 'Training_Images/thumbnail/'
check_shape = False

#abs_labels_dir = root + 'Completed_Labels.json'
abs_labels_dir = root + 'Training_Annotations/coco_small.json'

# Open the json file containing the absolute labels
with open(abs_labels_dir) as abs_labels_json:
    abs_labels_dict = json.load(abs_labels_json)

# Iterate through each image in abs_labels, extracting filename and id
for file in [{'name': image['file_name'].split('/')[-1], 'id': image['id']} for image in abs_labels_dict['images'][:]]:
    if file['name'].endswith('.png') or file['name'].endswith('.jpg'):
        # Read in the image
        dataset_img = cv2.imread(directory + file['name'])
        
        # Extract the bounding boxes for each screw in the image
        label_ids = []
        for index, abs_label in enumerate(abs_labels_dict['annotations'][:]):
            # Find the annotations that match the image
            # I want to know which images have annotations of area zero
            if abs_label['image_id'] == file['id']:

                # Read in absolute bounding box coordinates
                coordinates = abs_label['bbox']

                #Display image with bounding box
                start_point = (int(coordinates[0]), int(coordinates[1]))
                end_point = (int(coordinates[0] + coordinates[2]), int(coordinates[1] + coordinates[3]))
                
                print(start_point, end_point)
                print('\n')

                if abs_label['category_id'] == 2:
                    colour = (0, 0, 255)
                if abs_label['category_id'] == 4:
                    colour = (255, 0, 0)
                # draw the rectangle
                cv2.rectangle(dataset_img, start_point, end_point, color=colour, thickness= 1, lineType=cv2.LINE_8)

                print("Image details: {}".format(file))
                print("Label details: {}".format(abs_label))

        print("Zero area bounding box found in image {}".format(file['name']))
        # display the output
        print("Dimensions {}: {}".format(file['name'], dataset_img.shape))
        cv2.imshow('imageRectangle', dataset_img)
        if cv2.waitKey(0) == 27: break
        else: continue
    else:
        continue

cv2.destroyAllWindows()

(158, 201) (223, 256)


Image details: {'name': '000001.jpg', 'id': 1}
Label details: {'id': 1, 'image_id': 1, 'category_id': 2, 'area': 3575, 'bbox': [158, 201, 65, 55]}
Zero area bounding box found in image 000001.jpg
Dimensions 000001.jpg: (480, 640, 3)
(212, 61) (243, 94)


Image details: {'name': '000002.jpg', 'id': 2}
Label details: {'id': 2, 'image_id': 2, 'category_id': 2, 'area': 1023, 'bbox': [212, 61, 31, 33]}
(209, 521) (237, 543)


Image details: {'name': '000002.jpg', 'id': 2}
Label details: {'id': 3, 'image_id': 2, 'category_id': 2, 'area': 616, 'bbox': [209, 521, 28, 22]}
Zero area bounding box found in image 000002.jpg
Dimensions 000002.jpg: (640, 480, 3)
(86, 310) (111, 334)


Image details: {'name': '000003.jpg', 'id': 3}
Label details: {'id': 4, 'image_id': 3, 'category_id': 2, 'area': 600, 'bbox': [86, 310, 25, 24]}
(301, 199) (326, 226)


Image details: {'name': '000003.jpg', 'id': 3}
Label details: {'id': 5, 'image_id': 3, 'category_id': 2, 'area': 675, 'bbox': [

In [ ]:
# Convert Cross-Screw dataset to 480x640 dimensions

for infile in glob.glob(directory + "*.jpg"):
    file, ext = os.path.splitext(infile)
    file_name = os.path.basename(file)
    with Image.open(infile) as im:
        # When the image is in portrait mode, scale by flipped dimensions
        if im.size[0] > im.size[1]:
            im.thumbnail((640, 480))
        else:
            im.thumbnail((480, 640))
        
        im.save(directory + 'thumbnail/' + file_name + '.jpg', "JPEG")

In [12]:
# Update labels to match new image dimensions
with open(root + 'Training_Annotations/coco.json') as labels_json:
    labels_dict = json.load(labels_json)

for image in labels_dict['images']:
    old_width = image['width']
    old_height = image['height']

    if image['width'] > old_height:
        image['width'] = 640
        image['height'] = 480
    else:
        image['width'] = 480
        image['height'] = 640
    
    # Update the annotations
    for annotation in labels_dict['annotations']:
        if annotation['image_id'] == image['id']:
            annotation['bbox'][0] = int(annotation['bbox'][0] * image['width'] / old_width)
            annotation['bbox'][1] = int(annotation['bbox'][1] * image['height'] / old_height)
            annotation['bbox'][2] = int(annotation['bbox'][2] * image['width'] / old_width)
            annotation['bbox'][3] = int(annotation['bbox'][3] * image['height'] / old_height)
            annotation['area'] = annotation['bbox'][2] * annotation['bbox'][3]

with open(root + 'Training_Annotations/coco_small.json', 'w') as labels_json:
    json.dump(labels_dict, labels_json, indent=4)

Formatting dataset 2
Removing images that don't have a corresponding annotation file

In [18]:
# With yolo dataset
# Iterate through the dataset2 directory, deleting the images that do not have 
project_folder = os.path.abspath('C:/Users/drago/Documents/gitrepos/recycle_robot/')

dataset_2 = os.path.join(project_folder, "Autogathered_Dataset/2 (labeless removed)/Images/")

# Read in the coco.json file to remove the images that do not have labels
with open(project_folder + 'Autogathered_Dataset/2 (labeless removed)/coco.json') as labels_json:
    img_labels_dict = json.load(labels_json)

Total images: 687
Removed Images: 0


In [ ]:

count = 0
removed_count = 0
for file in os.listdir(dataset_2):
    if file.endswith(".png"):
        # Check if a file by the same name of .txt exists
        filename = os.path.splitext(os.path.join(dataset_2, file))[0]
        txt_file = filename + ".txt"
        #print(txt_file)
        if not os.path.isfile(txt_file):
            print(f"{filename} does not exist as .txt ... deleting: {txt_file}")
            
            #os.remove(os.path.join(dataset_2, file))
            removed_count += 1
            continue
        else:
            count += 1

print(f"Total images: {count}")
print(f"Removed Images: {removed_count}")